# Data Preprocessing :

We are going to preprocess the data to train are models

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import sys
import warnings
from bs4 import BeautifulSoup
warnings.filterwarnings("ignore")

In [2]:
df=pd.read_csv("train.csv")
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


### Data Cleaning:
 We clean the text by removing various tags, stopwords, special characters and converting them to lowercase.

In [3]:
'''##########[!] Functions reused in Django###############'''
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
'''def stemming(sentence):
    stemmer=SnowballStemmer("english")
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence'''
def clear_sentance(sentance):
    sentance= re.sub(r"http\S+", "", sentance)
    sentance = BeautifulSoup(sentance, 'lxml').get_text()
    sentance = decontracted(sentance)
    sentance = cleanPunc(sentance)
    sentance = re.sub("\S*\d\S*", "", sentance).strip()
    sentance = re.sub('[^A-Za-z]+', ' ', sentance)
    #sentance = stemming(sentance)
    # https://gist.github.com/sebleier/554280
    #https://towardsdatascience.com/journey-to-the-center-of-multi-label-classification-384c40229bff
    stop_words = set(stopwords.words('english'))
    stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
    sentance = ' '.join(e.lower() for e in sentance.split() if e.lower() not in  stopwords.words('english'))
    return sentance.strip()
    


In [4]:
from tqdm import tqdm
preprocessed_text = []
for sentance in tqdm(df['comment_text'].values):
    preprocessed_text.append(clear_sentance(sentance))


100%|██████████████████████████████████████████████████████████████████████████| 159571/159571 [31:57<00:00, 83.21it/s]


In [5]:
print(preprocessed_text[0])

explanation edits made username hardcore metallica fan reverted vandalisms closure gas voted new york dolls fac please remove template talk page since retired


In [6]:
df['preprocessed_text']=preprocessed_text
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,preprocessed_text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanation edits made username hardcore metal...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,daww matches background colour seemingly stuck...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man really trying edit war guy constantly ...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,make real suggestions improvement wondered sec...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,sir hero chance remember page


In [8]:
df.drop(['comment_text','id'],axis=1,inplace=True)
df.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,preprocessed_text
0,0,0,0,0,0,0,explanation edits made username hardcore metal...
1,0,0,0,0,0,0,daww matches background colour seemingly stuck...
2,0,0,0,0,0,0,hey man really trying edit war guy constantly ...
3,0,0,0,0,0,0,make real suggestions improvement wondered sec...
4,0,0,0,0,0,0,sir hero chance remember page


In [9]:
df[df['toxic']==1].head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,preprocessed_text
6,1,1,1,0,1,0,cocksucker piss around work
12,1,0,0,0,0,0,hey talk exclusive group wp talibans good dest...
16,1,0,0,0,0,0,bye look come think comming back tosser
42,1,0,1,0,1,1,gay antisemmitian archangel white tiger meow g...
43,1,0,1,0,1,0,fuck filthy mother ass dry


In [10]:
df.to_csv('preprocessed_text.csv',index=False)
df=pd.read_csv('preprocessed_text.csv')
df.dropna(axis=0,inplace=True)
df.isna().sum()

toxic                0
severe_toxic         0
obscene              0
threat               0
insult               0
identity_hate        0
preprocessed_text    0
dtype: int64

**Note:** Removed null values if any present.

In [11]:
df.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,preprocessed_text
0,0,0,0,0,0,0,explanation edits made username hardcore metal...
1,0,0,0,0,0,0,daww matches background colour seemingly stuck...
2,0,0,0,0,0,0,hey man really trying edit war guy constantly ...
3,0,0,0,0,0,0,make real suggestions improvement wondered sec...
4,0,0,0,0,0,0,sir hero chance remember page


In [12]:
df.to_csv('preprocessed_text.csv',index=False)